# Retrieval Augmented Generation (RAG) with Azure AI Search and OpenAI

This code demonstrates how to work with RAG to give more context to the LLM/SLM models to get a more accurate answer. The code uses Azure AI Search to index the documents and Azure OpenAI's embedding model to generate embeddings/vectors for the documents.

+ Create an index schema
+ Load the sample data from a local folder
+ Embed the documents in-memory using Azure OpenAI's text-embedding-ada-002 model
+ Index the vector and non-vector fields on Azure AI Search
+ Run a series of vector and hybrid queries, including metadata filtering and hybrid (text + vectors) search. 

The code uses Azure OpenAI to generate embeddings for title and content fields. You'll need access to Azure OpenAI to run this demo.

## Create the resources

Refer to the `README.md` file in the root folder to create the resources.

## Install python packages

In [1]:
%pip install python-dotenv
%pip install tiktoken
%pip install azure-search-documents
%pip install azure-identity
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Connect to the Azure AI Search and OpenAI

Load environment variables from the `.env` file

In [2]:
import os
import re
from openai import AzureOpenAI
from dotenv import load_dotenv
from dotenv import dotenv_values

if os.path.exists(".env"):
    load_dotenv(override=True)
    config = dotenv_values(".env")

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_chat_completions_deployment_name = os.getenv("AZURE_OPENAI_CHAT_COMPLETIONS_DEPLOYMENT_NAME")

azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
embedding_vector_dimensions = os.getenv("EMBEDDING_VECTOR_DIMENSIONS")

azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
azure_search_service_admin_key = os.getenv("AZURE_SEARCH_SERVICE_ADMIN_KEY")
search_index_name = os.getenv("SEARCH_INDEX_NAME")

openai_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_api_key,
    api_version="2024-06-01"
)

# Test connection to OpenAI ChatGPT
completion = openai_client.chat.completions.create(
    model=azure_openai_chat_completions_deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who are you ?"}
    ])
print(completion.to_json())

{
  "id": "chatcmpl-B4TIwttv1NHUo1N7PGxOcN4kpYyWW",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "I’m your helpful assistant, a conversational AI designed to provide information, answer questions, and assist with a variety of tasks. Think of me as your virtual companion, here to make your life easier and provide support whenever you need it! How can I assist you today?",
        "refusal": null,
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1740405658,
  "model": "gpt-4o-2024-11-20",
  "object

## Count the number of tokens in a text

Like LLM models, Embedding models defines a `max input`. It is defined in number of `tokens`. The `max_input` for `text-embedding-3-large` is 8191 tokens. So we need to split the text into chunks of 8191 tokens or less. For that, you need to get the number of tokens in a text string.

In [3]:
import tiktoken

def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name="cl100k_base")
    num_tokens = len(encoding.encode(string, disallowed_special=()))
    return num_tokens

# Test the function
num_tokens_from_string("tiktoken is great!")

6

The OpenAI embedding model `text-embedding-3-large` has a limit of `8191` tokens per request.
Before sending the files to the model, we need to split the text into chunks of less than `8191` tokens.
Count the number of tokens in the sample files and show the files with more than `8191` tokens.

In [7]:
#!pip install PyPDF2

In [8]:
import os
import PyPDF2

input_directory = './data/azure-ai-docs/'
i = 0

def num_tokens_from_string(string):
    # Assuming this function calculates tokens (you can define it as needed)
    return len(string.split())

for filename in os.listdir(input_directory):
    if filename.endswith('.pdf'):
        file_path = os.path.join(input_directory, filename)
        with open(file_path, 'rb') as file:  # Open in binary mode
            reader = PyPDF2.PdfReader(file)
            content = ''
            for page in reader.pages:
                content += page.extract_text() or ''  # Extract text from each page
            tokens = num_tokens_from_string(content)
            if tokens > 8191:
                print(f'File {filename} has {tokens} tokens which is more than 8191 (max) tokens')

File FY24-MER-2.7-Indicator-Reference-Guide.pdf has 102834 tokens which is more than 8191 (max) tokens


## Transforming/cleaning the documents

Later in this lab, we will proceed with markdown `.md` files. We will need to remove all special characters and markdown syntax from the files. The function `clean_markdown_content()` will help us with this.

In [44]:
import os
import re
import PyPDF2

# Function to count tokens (assuming it's word count for simplicity)
def num_tokens_from_string(string):
    return len(string.split())

# Function to clean extracted PDF content
def clean_pdf_content(content):
    # Remove links
    link_pattern = r'\[([^\[]+)\]\(([^\)]+)\)'
    content = re.sub(link_pattern, r'\1', content)

    # Remove images
    image_pattern = r'\!\[([^\[]*)\]\(([^\)]+)\)'
    content = re.sub(image_pattern, '', content)

    # Remove [doc][doc] references
    doc_reference_pattern = r'\[doc\]\[doc\]'
    content = re.sub(doc_reference_pattern, '', content)

    # Remove all occurrences of ** and newlines
    content = content.replace('**', '')
    content = content.replace('\n', ' ').strip()

    return content

# Path to the directory with PDFs
input_directory = './data/azure-ai-docs/'
i = 0

# Loop through all PDFs and process
for filename in os.listdir(input_directory):
    if filename.endswith('.pdf'):
        file_path = os.path.join(input_directory, filename)
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            content = ''
            for page in reader.pages:
                extracted_text = page.extract_text() or ''
                cleaned_text = clean_pdf_content(extracted_text)
                content += cleaned_text + ' '  # Add a space between pages

            tokens = num_tokens_from_string(content)
            if tokens > 8191:
                print(f'File {filename} has {tokens} tokens, exceeding the limit of 8191.')

File FY24-MER-2.7-Indicator-Reference-Guide.pdf has 102833 tokens, exceeding the limit of 8191.


## Get the vector embedding for an input text

In [45]:
def get_embeddings_vector(text):

    response = openai_client.embeddings.create(
        input=text,
        model=azure_openai_embedding_model,
    )

    embedding = response.data[0].embedding

    return embedding

# Test the function
vector = get_embeddings_vector("Sample text")
print(vector)

[-0.012435130774974823, -0.04316585138440132, -0.009822873398661613, 0.011554595082998276, 0.006599131505936384, -0.013384154066443443, -0.04163958877325058, 0.059954747557640076, -0.019371801987290382, 0.0006316626095212996, 0.028959864750504494, 0.007949287071824074, 0.008849390782415867, -0.05157986655831337, 0.013932042755186558, 0.013256965205073357, -0.010253357701003551, 0.00492366636171937, 0.008017773739993572, -0.02305048704147339, -0.002491184277459979, 0.004666842985898256, -0.026592200621962547, 0.051892947405576706, 0.007430749014019966, -0.006525753531605005, -0.01613338477909565, 0.012797129340469837, 0.007919936440885067, 0.024635452777147293, 0.008986363187432289, 0.03978068009018898, -0.005650108680129051, -0.028294570744037628, 0.01490063313394785, 0.013755936175584793, 0.03093617968261242, 0.020193634554743767, 0.027844518423080444, 0.007939503528177738, 0.026044311001896858, 0.015389819629490376, -0.04449643939733505, -0.015487657859921455, -0.02007623016834259, 0

## Create file chunks

This is where we split the markdown files in folder `./data/azure-ai-docs` into chunks.

In [50]:
import uuid
import re
import json
import os
import fitz  # PyMuPDF for PDF reading
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

input_directory = './data/azure-ai-docs/'
output_directory = './data/chunks/'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

chunk_index = 0

def clean_content(text):
    """Remove unnecessary characters and clean text."""
    text = re.sub(r'\[doc\]\[doc\]', '', text)  # Remove redundant doc references
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    return text.strip()

def split_into_chunks(text, max_tokens=500):
    """Split text into meaningful chunks based on sentence boundaries."""
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_length = 0

    for sentence in sentences:
        sentence_tokens = len(sentence.split())
        if current_length + sentence_tokens <= max_tokens:
            current_chunk += " " + sentence
            current_length += sentence_tokens
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_length = sentence_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Loop through each file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.pdf'):
        # Open and extract text from the PDF
        with fitz.open(os.path.join(input_directory, filename)) as doc:
            print(f"Processing {filename}")
            content = ""
            for page in doc:
                content += page.get_text()  # Extract text from each page

            # Preview extracted content for debugging
            print("Extracted Content Preview:")
            print(content[:100])  # Print the first 100 characters for better preview

            # Check if there is any substantial content
            if len(content.strip()) == 0:
                print(f'No extractable text found in {filename}')
                continue

            # Clean and normalize content
            content = clean_content(content)

            # Split content into optimized chunks
            chunks = split_into_chunks(content)
            print(f"Total Chunks Generated: {len(chunks)}")

            # Process and save each chunk
            for chunk in chunks:
                chunk_content = chunk.strip()
                if len(chunk_content) == 0:
                    continue  # Skip empty chunks

                chunk_index += 1

                # Embedding generation
                vector = get_embeddings_vector(chunk_content)

                chunk_data = {
                    "id": str(uuid.uuid4()),
                    'filename': filename,
                    'chunk_index': chunk_index,
                    'chunk_content': chunk_content,
                    'vector': vector
                }

                chunk_file_name = f'chunk_{chunk_index}_{filename}.json'.replace('?', '').replace(':', '').replace("'", '').replace('|', '').replace('/', '').replace('\\', '')

                # Write chunk into JSON file
                with open(f'{output_directory}/{chunk_file_name}', 'w') as f:
                    json.dump(chunk_data, f)

print("Chunk extraction complete!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\georg\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Processing FY24-MER-2.7-Indicator-Reference-Guide.pdf
Extracted Content Preview:
 1 
  
 
 
 
MER 2.0 (Version 2.7)  
September 2023 
Monitoring, Evaluation, and 
Reporting Indicato
Total Chunks Generated: 205
Chunk extraction complete!


By default, the length of the embedding vector will be `1536` for `text-embedding-3-small` or `3072` for `text-embedding-3-large`. You can reduce the dimensions of the embedding by passing in the dimensions parameter without the embedding losing its concept-representing properties.

## Create Index in Azure AI Search.

In [51]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    SearchField,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticSearch,
    SemanticField,
    LexicalAnalyzerName
)

# Set credentials and initialize client
credential = AzureKeyCredential(azure_search_service_admin_key)

search_index_client = SearchIndexClient(
    endpoint=azure_search_service_endpoint, 
    index_name=search_index_name, 
    credential=credential
)

# 🔥 Delete existing index if it already exists
try:
    existing_index = search_index_client.get_index(search_index_name)
    if existing_index:
        search_index_client.delete_index(search_index_name)
        print(f"Deleted existing index: {search_index_name}")
except Exception as e:
    print(f"No existing index found or error occurred: {e}")

# ✅ Define fields for the new index with lowercase normalization
fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True,
        facetable=True,
    ),
    SearchableField(name="page_title", type=SearchFieldDataType.String, analyzer_name=LexicalAnalyzerName.EN_LUCENE),
    SearchableField(name="page_description", type=SearchFieldDataType.String, analyzer_name=LexicalAnalyzerName.EN_LUCENE),
    SearchableField(name="page_date", type=SearchFieldDataType.String, analyzer_name=LexicalAnalyzerName.EN_LUCENE),
    SearchableField(name="chunk_title", type=SearchFieldDataType.String, analyzer_name=LexicalAnalyzerName.EN_LUCENE),
    SearchableField(name="chunk_content", type=SearchFieldDataType.String, analyzer_name=LexicalAnalyzerName.EN_LUCENE),
    SearchField(
        name="vector", 
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=3072,  # Set dimension
        vector_search_profile_name="myHnswProfile",
    ),
]

# ✅ Configure vector search
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw"
        )
    ]
)

# ✅ Configure semantic search
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="page_title"),
        content_fields=[SemanticField(field_name="chunk_content")]
    )
)

semantic_search = SemanticSearch(configurations=[semantic_config])

# ✅ Define and create the search index
search_index = SearchIndex(
    name=search_index_name, 
    fields=fields, 
    vector_search=vector_search, 
    semantic_search=semantic_search
)

# ✅ Create or update the index on Azure
result = search_index_client.create_or_update_index(search_index)
print(f"{result.name} created successfully.")

Deleted existing index: index-doc
index-doc created successfully.


## Upload chunks/documents to Azure AI Search

In [52]:
import uuid
import os
import json
from azure.search.documents import SearchClient
from azure.core.exceptions import HttpResponseError

# Initialize the SearchClient
search_client = SearchClient(
    endpoint=azure_search_service_endpoint,
    index_name=search_index_name,
    credential=credential
)

# Define batch size to prevent 'Request Entity Too Large' error
BATCH_SIZE = 500  # Adjust if necessary based on document size

# Define allowed fields based on your index schema
allowed_fields = {
    "id", "page_title", "page_description", "page_date", "chunk_title", "chunk_content", "vector"
}

# Function to clean and filter documents based on allowed fields
def clean_document(doc):
    return {key: value for key, value in doc.items() if key in allowed_fields}

# Function to split documents into manageable batches
def batch_documents(documents, batch_size):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

# Prepare documents for upload
documents_batch = []

for filename in os.listdir(output_directory):
    if filename.endswith('.json'):
        with open(os.path.join(output_directory, filename), 'r') as file:
            document = json.load(file)
            cleaned_doc = clean_document(document)  # Clean document
            documents_batch.append(cleaned_doc)

# Upload documents in batches
try:
    if documents_batch:
        for batch_number, batch in enumerate(batch_documents(documents_batch, BATCH_SIZE), 1):
            result = search_client.upload_documents(documents=batch)
            for doc_result in result:
                print(f"Batch {batch_number} - Upload of document ID {doc_result.key} succeeded: {doc_result.succeeded}")
    else:
        print("No documents found for upload.")
except HttpResponseError as e:
    print(f"An error occurred during document upload: {e}")


Batch 1 - Upload of document ID 7ea77e34-3558-4fa3-a9fe-9c1573f9ac8f succeeded: True
Batch 1 - Upload of document ID 7b77e52a-c65d-4d37-bda6-1d82ab897241 succeeded: True
Batch 1 - Upload of document ID a3248eb2-ab99-4603-aa67-350292aeac3c succeeded: True
Batch 1 - Upload of document ID bcba2034-860b-4acb-934c-6acd36487e0e succeeded: True
Batch 1 - Upload of document ID 5d500309-7825-4c98-ba5f-cfbdded67bf3 succeeded: True
Batch 1 - Upload of document ID 96650e41-beaf-4360-96c1-e1c4173c9c92 succeeded: True
Batch 1 - Upload of document ID dde0909d-71c9-4b54-b871-de89fe172c4c succeeded: True
Batch 1 - Upload of document ID 64f32b07-b636-45e6-affb-8c3367db1b66 succeeded: True
Batch 1 - Upload of document ID 15c0ad9e-0bba-472e-b216-af678c080017 succeeded: True
Batch 1 - Upload of document ID 10921f06-59f7-4259-9eef-551162de76df succeeded: True
Batch 1 - Upload of document ID aea03d8d-aebf-452e-8980-9b935502f098 succeeded: True
Batch 1 - Upload of document ID a6272f6c-b2e2-4812-a7bc-90e0ac252

## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [54]:
from azure.search.documents.models import VectorizedQuery

# Pure Vector Search
query = "Explain what is TX_CURR ?"  

embedding = get_embeddings_vector(query)

vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=[
            #page_title", "page_date", 
            "chunk_title", 
            "chunk_content"],
)  
  
for result in results:
    print(f"-------------------------------------------")
    #print(f"Page Date: {result['page_date']}")  
    #print(f"Page Title: {result['page_title']}")  
    print(f"Chunk Title: {result['chunk_title']}")  
    print(f"Chunk Content: {result['chunk_content']}")
    print(f"Score: {result['@search.score']}")  


-------------------------------------------
Chunk Title: None
Chunk Content: On an annual basis? 3. Describe differences in MMD uptake across age and sex groups and sites/SNUs. Data visualization & use examples: *See next page* TREATMENT 153 95-95-95 Cascade Example: Population, PLHIV, and TX_CURR by Five-Year Age Band: TREATMENT 154 TX_ML Description: Number of ART patients (who were on ART at the beginning of the quarterly reporting period or initiated treatment during the reporting period) and then had no clinical contact since their last expected contact Numerator: Number of ART patients (currently on ART or newly initiating ART) with no clinical contact or ARV pick-up for greater than 28 days since their last expected clinical contact or ARV pick-up Clinical contact is defined as any clinical interaction with the patient, such as clinical assessment by a healthcare worker or provision of medication. Denominator: N/A Indicator changes (MER v2.6.1 to v2.7): None Reporting level: Fac

## Simulate a user query

This is where we will use the Azure AI Search to search for documents similar to the user query.

In [55]:
response = openai_client.chat.completions.create(
    model=azure_openai_chat_completions_deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant for an AI learner."},
        {"role": "user", "content": "Explian the meaning of TX_CURR Indicator?"}
    ],
    extra_body={
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": azure_search_service_endpoint,
                    "index_name": search_index_name,
                    "authentication": {
                        "type": "api_key",
                        "key": azure_search_service_admin_key,
                    }
                }
            }
        ]
    }
)

print(response.to_json())

{
  "id": "4669435e-0ada-4b51-909f-7bc6f3a42ab1",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The **TX_CURR** indicator refers to the number of people living with HIV (PLHIV) who are currently on antiretroviral therapy (ART) at the end of a reporting period. It includes patients who were already on ART at the beginning of the reporting period and those who newly initiated ART during the reporting period. \n\nKey points about TX_CURR:\n1. **Definition of Clinical Contact**: Clinical contact is defined as reporting to the clinic for ART pick-up, clinical assessment, or a documented community visit with a community health worker or peer from an ART refill group [doc5].\n2. **Exclusions**: Patients who were already lost to follow-up and not counted in TX_CURR at the beginning of the reporting period should not be included in this indicator [doc5].\n3. **Re-engagement**: If a patient who was off treatment for more than 28 days

In [56]:
print(response.choices[0].message.content)

The **TX_CURR** indicator refers to the number of people living with HIV (PLHIV) who are currently on antiretroviral therapy (ART) at the end of a reporting period. It includes patients who were already on ART at the beginning of the reporting period and those who newly initiated ART during the reporting period. 

Key points about TX_CURR:
1. **Definition of Clinical Contact**: Clinical contact is defined as reporting to the clinic for ART pick-up, clinical assessment, or a documented community visit with a community health worker or peer from an ART refill group [doc5].
2. **Exclusions**: Patients who were already lost to follow-up and not counted in TX_CURR at the beginning of the reporting period should not be included in this indicator [doc5].
3. **Re-engagement**: If a patient who was off treatment for more than 28 days is re-engaged and restarted on ART, they can be added back to TX_CURR, provided they remain on treatment until the end of the reporting period. However, they shoul